In [1]:
%run supportvectors-common.ipynb



<center><img src="https://d4x5p7s4.rocketcdn.me/wp-content/uploads/2016/03/logo-poster-smaller.png"/> </center>
<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# Getting started with `LangChain`

### Learning goals

We will learn the basics of:
    
* what is a `langchain`, and what purpose it serves
* a simple prompt template
* a simple agent that does something useful

## Environment setup

There are two steps needed to get started:

* install the necessary libraries
* get the necessary api keys, and set them as environment variables on our workstation.

#### Library installation

Run the following cell only once, to install the libraries. After that, comment it out.


In [9]:
#
# Uncomment this only if needed.
#
!pip install langchain --upgrade
!pip install google-search-results --upgrade


#### Needed Imports
Let us now import the necessary components for this lab

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType
from IPython.display import Markdown, HTML

display(HTML('<b> Libraries installed, and components imported </b>'))

## OpenAI and SERP API keys

In order to be able to use some of the large language models, let us register with the `openai` website and get an api key. Likewise, let's do the same for `serpapi`. The respective website are:

* **SerpApi**: https://serpapi.com/ Register at this site, and then login. The look for your developer api key, and copy it.

* **OpenAI**: https://platform.openai.com/account/api-keys Create a key, and be careful that you digitally note it down immediately -- you won't be able to see it later.

<div class="alert-box alert-warning" style="padding-top:30px">
   
<b >Caveat Emptor</b>

>Be careful not to share these keys publicly! They are associated with your account -- and after you have exhausted your free tokens, the respective api providers meter your usage, and charge you accordingly.
>
>For this reason, it is a particularly bad idea to hard-wire it in your jupyter notebooks: sooner or later, you will end up sharing your notebook more widely than you intended. In that eventuality, you incur the risk of raking of large bills from the api providers for use by unauthorized people on your behalf.
</div>


Then, we need to add two environment variables `OPENAI_API_KEY` on our workstation:

* `OPENAI_API_KEY` for the OpenAI api key
* `SERPAPI_API_KEY` for the SerpApi key


For Linux distributions and Mac, one way to accomplish this is to add the following line to one of the `~/.bashrc`,  `~/.bash_profile`, `~/.profile`, or `~/.zshrc` files, based on the shell and distribution you are using.

```
export OPENAI_API_KEY="<<your openai-api-key>>"
export SERPAPI_API_KEY="<<your serpapi key>>"
```

For MS-Windows, try setting these environment variables with the following steps:

* launch the control panel
* go to `System -> Advanced settings -> Environment variables`
* Add the new environment variables
* Quite likely, on windows, you will need to restart the jupyter kernel for the notebook to pick up the envronment variables.


#### Instantiate a large language model

Now, let us instantiate an `OpenAI` object that will use the large language model that the OpenAI provides as a remote api service. To help it produce rather varied results, we have set the `temperature` high. More about the meaning of temperature in the next lab -- for now, it suffices us to take it for granted that this value will work.

In [23]:
#!pip install OpenAI

In [13]:
llm = OpenAI(temperature=0.9)

#### Invoke the LLM to generate text

The `OpenAI` large language model is generative: so it responds with a textual output to input prompts.

In [14]:
text = "What is a good name for a cute mobius strip?"
print(llm(text))



Twisted Tail


Next time we make the same invocation, we should expect a different result:

In [15]:
text = "What is a good name for a cute mobius strip?"
print(llm(text))



Twisty Turnaround


## Prompt

A prompt is a template, with placeholders for user inputs. In the following, let us get a brief preview of how it works. We will start by creating a prompt, that expects a user-input called `book`.

In [16]:
prompt = PromptTemplate(
    input_variables=["book"],
    template="Who is the author of the book {book}?",
)

Now, we can have it produce text to go into the large language model by supplying a value for the input variable `book`:

In [17]:
text = prompt.format(book = 'A snail of two cities')
print (text)

Who is the author of the book A snail of two cities?


and finally, pass the output of the prompt to the large language model.

In [18]:
author = llm(text)
print (author)



The book A Snail of Two Cities was written by Lola M. Schaefer.


## LLM Chain

Now, we could create a useful chain by concatenating the two components, prompt and the large language model. When we do so, it become a reusable component, and we do not have to ferry the output of one component (the prompt here) to be the input to the second component (the large language model).

Let us see now we can create this very simple chain:

In [19]:
chain = LLMChain(llm=llm, prompt=prompt)
author = chain.run('A snail of two cities')
print(author)



The author of the book A Snail of Two Cities is Kenneth Baker.


## Agents

Agents in `langchain` provide an ability to call chains conditional on the user inputs. In other words, based on the context, the agent determines which tools and LLMs they need to use. This creates flexible and dynamic chains, thus providing us considerably powerful ways to engage with these models.

In other words, `langchain` derives its power from the interplay of three entities:

* **Large Language Models**: the large language models powering the agent
* **Tools**: a tool is simply a function that does something useful -- such as perform a google search, retrieve something from a database, run a `REPL`, or fire other chains. The basic signature of the function comprises a string as input, and a string as output. This leaves considerable latitude in implementing tools.
* **Agents**: agents handle the overall orchestration or flow control, based on context and user input



In [20]:
tool_names = ['serpapi',   # for google search
              'llm-math'  # this particular tool needs an llm too, so need to pass that
             ]

tools = load_tools (tool_names = tool_names, 
                    llm        = llm)

Now that we have the tools, and a large language model available, let us orchestrate them with an agent.

In [21]:
agent = initialize_agent (tools   = tools, 
                          llm     = llm,
                          agent   = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                          verbose = True
                         )

Let us start with the following prompt text:

> What is the height of mount everest in meters? If an aeroplane has to travel 1 km higher than that number, how high should it be?


In [22]:
question = """
What is the height of mount everest in meters today? 
If an aeroplane has to travel 1 km higher than
that number, how high should it be?
"""

agent.run(question)



> Entering new AgentExecutor chain...
 I should look up the current height of Mount Everest first
Action: Search
Action Input: "Height of Mount Everest"
Observation: 29,032′
Thought: I need to convert this to meters
Action: Calculator
Action Input: 29,032' in meters
Observation: Answer: 8848.9536
Thought: I now need to add 1 km
Action: Calculator
Action Input: 8848.9536 + 1000
Observation: Answer: 9848.9536
Thought: I now know the final answer
Final Answer: The aeroplane should be 9,848.9536 meters high to be 1 km higher than Mount Everest.

> Finished chain.


'The aeroplane should be 9,848.9536 meters high to be 1 km higher than Mount Everest.'

**Observations**

* note that we forced the agent to use the search tool, since we implied that the height of the Everest keeps changing, and we need to know the height today
* the search tool returned the height in imperial units (feet)
* so there was a need to first convert it to SI units (meter)
* finally, the math tool had to add a 1000 meters to the value to figure out the answer.

#### Homework

What would have happened if we did not specify `today` in the question? Try it out. Explain why it is that way.

In [24]:
question = """
What is the height of mount everest in meters? 
If an aeroplane has to travel 1 km higher than
that number, how high should it be?
"""

agent.run(question)



> Entering new AgentExecutor chain...
 I should find out the height of mount everest first
Action: Search
Action Input: "height of mount everest"
Observation: 29,032′
Thought: I can now use a calculator to figure out the height in meters
Action: Calculator
Action Input: 29,032 ft
Observation: Answer: 95249.34688
Thought: I need to add 1 km to that
Action: Calculator
Action Input: 95249.34688 + 1000
Observation: Answer: 96249.34688
Thought: I now know the final answer
Final Answer: The aeroplane should fly at a height of 96249.34688 meters.

> Finished chain.


'The aeroplane should fly at a height of 96249.34688 meters.'